## combinando conjunto de dados
---

In [1]:
import pandas as pd

#### .concat()
---

usando a função `.concat()` pode-se juntar duas ou mais series ou dataframes:

In [2]:
sr1 = pd.Series([i for i in range(0, 10, 2)], index=list('abcde'))
sr2 = pd.Series([i for i in range(11, 21, 2)], index=list('fghij'))
print(sr1)
print(sr2)

a    0
b    2
c    4
d    6
e    8
dtype: int64
f    11
g    13
h    15
i    17
j    19
dtype: int64


In [3]:
sr3 = pd.concat([sr1, sr2])
sr3

a     0
b     2
c     4
d     6
e     8
f    11
g    13
h    15
i    17
j    19
dtype: int64

observe que é necessário passar as datasets usando uma lista como parâmetro para `.concat()`. Serve para datafranes também:

In [4]:
df1 = pd.DataFrame([['a1', 'a2'], ['a3', 'b4']], columns=['1-2', '3-a'])
df2 = pd.DataFrame([['a1', 'b2'], ['b3', 'c4']], columns=['1-2', '3-a'])

In [5]:
df1

,1-2,3-a
0,a1,a2
1,a3,b4


In [6]:
df2

,1-2,3-a
0,a1,b2
1,b3,c4


In [7]:
df3 = pd.concat([df1, df2])
df3

,1-2,3-a
0,a1,a2
1,a3,b4
0,a1,b2
1,b3,c4


por padrão, ao concatenar dataframes, estes serão juntados através das linhas (`axis=0`). isto pode ser mudado usando `axis=1`:

In [8]:
df4 = pd.concat([df1, df2], axis=1)
df4

,1-2,3-a,1-2,3-a
0,a1,a2,a1,b2
1,a3,b4,b3,c4


se os dataframes tiverem índices explícitos, estes permanecem ao serem concatenados, o que pode gerar duplicação de informação:

In [9]:
df5 = pd.DataFrame([['a1', 'a2'], ['a3', 'b4']], columns=['1-2', '3-a'], index=list('ab'))
df6 = pd.DataFrame([['a1', 'b2'], ['b3', 'c4']], columns=['1-2', '3-a'], index=list('ab'))

In [10]:
df5

,1-2,3-a
a,a1,a2
b,a3,b4


In [11]:
df6

,1-2,3-a
a,a1,b2
b,b3,c4


In [12]:
df7 = pd.concat([df5, df6])
df7

,1-2,3-a
a,a1,a2
b,a3,b4
a,a1,b2
b,b3,c4


ou, ainda, falta de valores:

In [13]:
df8 = pd.DataFrame([['a1', 'a2'], ['a3', 'b4']], columns=['1-2', '3-a'], index=list('ab'))
df9 = pd.DataFrame([['a1', 'b2'], ['b3', 'c4']], columns=['1-2', '3-a'], index=list('cd'))

In [14]:
df8

,1-2,3-a
a,a1,a2
b,a3,b4


In [15]:
df9

,1-2,3-a
c,a1,b2
d,b3,c4


In [16]:
df10 = pd.concat([df8, df9], axis=1)
df10

,1-2,3-a,1-2,3-a
a,a1,a2,NaN,NaN
b,a3,b4,NaN,NaN
c,NaN,NaN,a1,b2
d,NaN,NaN,b3,c4


com o parâmetro `verify_integrity=True`, é apresentado uma exceção caso haja algum índice repetindo:

```
pd.concat([df8, df9], axis=1, verify_integrity=True)
```
o que apresenta a mensagem:
```
ValueError: Indexes have overlapping values
```

ou, se preferir, pode simplismente ignorar os índices repetidos e o próprio pandas completa com índices diferentes, basta usar `ignore_index=True`:

In [17]:
pd.concat([df8, df9], axis=1, ignore_index=True)

,0,1,2,3
a,a1,a2,NaN,NaN
b,a3,b4,NaN,NaN
c,NaN,NaN,a1,b2
d,NaN,NaN,b3,c4


pode, ainda, criar um multiindexing através do parâmetro `keys=` onde deve ser passado o nome dos níveis:

In [18]:
pd.concat([df8, df9], axis=1, keys=['1º', '2º'])

1º        2º     
   1-2  3-a  1-2  3-a
a   a1   a2  NaN  NaN
b   a3   b4  NaN  NaN
c  NaN  NaN   a1   b2
d  NaN  NaN   b3   c4

na situação em que os dataframes têm várias colunas e linhas e elas nem sempre são iguais, a concatenação acaba ficando com valores NA, como visto acima. Neste caso, pode se usar o parâmetro `join=`

In [19]:
dfd = pd.DataFrame([['a1', 'a2'], ['a3', 'b4']], columns=['1-2', '3-a'], index=list('ab'))
dff = pd.DataFrame([['a1', 'b2'], ['b3', 'c4']], columns=['1-2', '3-a'], index=list('bc'))

In [20]:
dfd

,1-2,3-a
a,a1,a2
b,a3,b4


In [21]:
dff

,1-2,3-a
b,a1,b2
c,b3,c4


In [22]:
pd.concat([dfd, dff], axis=1)

,1-2,3-a,1-2,3-a
a,a1,a2,NaN,NaN
b,a3,b4,a1,b2
c,NaN,NaN,b3,c4


In [23]:
pd.concat([dfd, dff], axis=1, join='outer')

,1-2,3-a,1-2,3-a
a,a1,a2,NaN,NaN
b,a3,b4,a1,b2
c,NaN,NaN,b3,c4


In [24]:
pd.concat([dfd, dff], axis=1, join='inner')

,1-2,3-a,1-2,3-a
b,a3,b4,a1,b2


`outer` é o valor padrão, por isso, não há mudanças entre usá-lo ou não. Já o `inner` apresenta apenas os valores de índices presentes em todos os dataframes.

usando `join_axes=`, pode ser passado uma lista contendo índices apenas os índices que deseja apresentar.

há, ainda, a opção de usar `<dataset1>.append(<dataset2>)`, onde dataset pode ser uma series ou dataframe:

In [25]:
dfd._append(dff)

,1-2,3-a
a,a1,a2
b,a3,b4
b,a1,b2
c,b3,c4


este método é menos eficiente, já que ele cria um novo dataset com as informações das outras, ao invés de simplesmente juntá-las, o que pode gastar muita memórias, principalmente em projetos grandes. Além disso, `._append()` tem menos funcionalidades que as demais opções.

#### .merge()
---

esta função trabalha de três formas ao juntas datasets: one-to-one joins, many-to-one joins e many-to-many joins. Cada forma são chamadas da mesma forma, usando a função `.merge()`, que reconhece com qual forma trabalhar altomaticamente, dependendo do tipo de dado que for passado.

a one-to-one joins é bastante similar às funções `.concat()` e `._append()`:

In [31]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],'hire_date': [2004, 2008, 2012, 2014]})

In [27]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [28]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [30]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


observe que, altomaticamente, a função `.merge()` reconhece as colunas que são comuns entre os datasets que estão sendo juntados e mescla estas colunas. É importante notar, também, que as informações que se repetem, mas não estão na mesma posição são corrigidas pela função, e que os índices, se for passados índices explícitos, são descartados, a não ser nos casos expeciais em que mescla por index, usando `left_index` ou `right_index`.

já o many-to-one joins ocorre quando há uma coluna que se repete entre os datasets. neste caso, cada entrada será preservada:

In [32]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],'supervisor': ['Carly', 'Guido', 'Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [34]:
df5 = pd.merge(df3, df4)
df5

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


neste caso, como, entre os dataframes, havia duas colunas `employee`, a função `.merge()` juntou as duas colunas em uma só, adicionou a colunas `supervisor` e utilizou a coluna `employee` que já exitia para preencher para completar os dados, por isso, o personagem `Guido` ficou repetido.

o many-to-many joins junta novamente as várias colunas que se repetem entre os datasets, sendo uma versão mais ampla do many-to-one, onde aquela mescla ocorre com várias colunas ao mesmo tempo:

In [35]:
df6 = pd.DataFrame({'group': ['Accounting', 'Accounting','Engineering', 'Engineering', 'HR', 'HR'],'skills': ['math', 'spreadsheets', 'coding', 'linux','spreadsheets', 'organization']})
df6

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [38]:
df7 = pd.merge(df1, df6)
df7

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


usando o parâmetro `on=` é possível especificar qual coluna deve ser mesclada, ou, ainda, passando uma lista de nome de coluna, quais colunas devem ser mescladas. só funciona se existir o mesmo nome nos diferentes datasets:

In [39]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


se as colunas, entre as datasets, tiverem nomes diferentes mas informações iguais, deve-se usar os parâmetros `left_on=` e `right_on=`:

In [40]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],'salary': [70000, 80000, 120000, 90000]})
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [42]:
df8 = pd.merge(df1, df3, left_on="employee", right_on="name")
df8

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


assim, para apagar a coluna repetida, basta usar a função `.drop('<coluna>', axis=)`:

In [44]:
df8.drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


se desejar mesclar dois datasets pelas linhas, é possível fazer isso usando os parâmetros `left_index=` e `right_index=`, que devem receber o valor booleano `True`:

In [45]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [46]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [47]:
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [53]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


também serve `<dataset1>.join(<dataset2>)`:

se desejar misturar colunas e linhas, deve-se combinar `left_index=` com `right_on=` ou `left_on=` com `right_index=`

In [56]:
pd.merge(df1a, df3, left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


desta forma, pode ser juntado múltiplos índices e colunas.

quando, entre dois datasets, há informações que aparecem em apenas uma coluna, mas não na outra, o seguinte pode ocorrer:

In [57]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],'food': ['fish', 'beans', 'bread']},columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],'drink': ['wine', 'beer']},columns=['name', 'drink'])

In [58]:
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [59]:
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [60]:
pd.merge(df6, df7)

,name,food,drink
0,Mary,bread,wine


isto ocorre porque só há uma informação compatível entre os dataframes. este comportamento pode ser especificado pelo perâmetro `how=`, que, por padrão, é `inner`.

se colocar `outer`, todas as informações são mescladas, e aquelas que não se complementam são preenchidas com NA:

In [61]:
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


com `left`, a mescla preenche com a informações do primeiro dataframe passado; já com `right`, preenche com o segundo:

In [62]:
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [63]:
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


quando, entre dois datasets, há colunas com mesmo nome, mas com informações diferentes, `.merge()` preserva as duas colunas e altomaticamente nomea-as para que sejam únicas:

In [64]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],'rank': [3, 1, 4, 2]})

In [65]:
df8

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [66]:
df9

,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [68]:
pd.merge(df8, df9, on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


nestes casos, é possível nomear como que deve ser feito a difereciação entre os nomes das colunas, usando `suffixes=`:

In [69]:
pd.merge(df8, df9, on='name', suffixes=[' inicial', ' final'])

,name,rank inicial,rank final
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2
